In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document

from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
import json


# .env 파일에서 환경변수 로드
load_dotenv()
api_key = os.getenv("OPEN_AI_KEY")

llm = ChatOpenAI(model="gpt-4o", api_key=api_key)

In [4]:
ml_txt_list = []
dl_txt_list = []
llm_txt_list = []
python_txt_list = []
open_source_txt_list = []

In [5]:
import os
# 파일을 읽어와서 text_list에 저장하는 함수
def load_files_to_list(file_path, text_list):
    if os.path.exists(file_path):  # 파일이 존재하는지 확인
        with open(file_path, 'r', encoding='utf-8') as file:  # 파일 열기
            content = file.read()  # 파일 내용 읽기
            text_list.append(content)  # text_list에 추가
    else:
        print(f"File {file_path} not found.")

def save_txt_list(type_:str, length_:int, list_:list):
    file_paths = [f"dataset/{type_}_dataset_{i}.txt" for i in range(1, length_+1)]  # 파일 경로 목록 (DL_dataset_1.txt, DL_dataset_2.txt, ...)

    for j in range(len(file_paths)):
        load_files_to_list(file_paths[j], list_)

    print(f"{type_} 관련 {len(list_)} 개의 파일 저장 완료")

    return list_

In [6]:
ml_txt_list = save_txt_list("ML", 23, ml_txt_list)
dl_txt_list = save_txt_list("DL", 16, dl_txt_list)
llm_txt_list = save_txt_list("LLM", 18, llm_txt_list)
python_txt_list = save_txt_list("PYTHON", 16, python_txt_list)
open_source_txt_list = save_txt_list("OPENSOURCE", 7, open_source_txt_list)

ML 관련 23 개의 파일 저장 완료
DL 관련 16 개의 파일 저장 완료
LLM 관련 18 개의 파일 저장 완료
PYTHON 관련 16 개의 파일 저장 완료
OPENSOURCE 관련 7 개의 파일 저장 완료


In [117]:
from langchain_openai import OpenAIEmbeddings
import random
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document
import json
from pprint import pprint


def get_retriever(texts: str, current_index:int):

    # text_list를 Document 객체로 변환
    documents = [Document(page_content=texts)]

    recursive_text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=20,
        length_function=len,
        is_separator_regex=False,
    )

    splits_recur = recursive_text_splitter.split_documents(documents)
    total_chunks = len(splits_recur)
    # 다음 인덱스 계산
    next_index = current_index + 10
    if next_index > total_chunks:  # 초과 시 순환 처리
        selected_splits = splits_recur[current_index:] + splits_recur[:next_index % total_chunks]
    else:
        selected_splits = splits_recur[current_index:next_index]

    # # current_index 업데이트 (다음 호출을 위한 값)
    # current_index = next_index % total_chunks
    splits = selected_splits


    print("Top 10 chunks:")
    for i, chunk in enumerate(splits[:10], 1):
        pprint(f"\nChunk {i}:\n{chunk.page_content}")

    # OpenAI 임베딩 모델 초기화
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)
    vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
    bm25_retriever = BM25Retriever.from_documents(splits)
    faiss_retriever = vectorstore.as_retriever()

    retriever = EnsembleRetriever(
                retrievers=[bm25_retriever, faiss_retriever],
                weights=[0.5, 0.5],  # 가중치 설정 (가중치의 합은 1.0)
            )

    return retriever

In [80]:
from langchain_openai import OpenAIEmbeddings
import random
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document
import json
from pprint import pprint


def get_retriever(texts: str):

    # text_list를 Document 객체로 변환
    documents = [Document(page_content=texts)]

    recursive_text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=20,
        length_function=len,
        is_separator_regex=False,
    )

    splits_recur = recursive_text_splitter.split_documents(documents)
    splits = splits_recur


    print("Top 10 chunks:")
    for i, chunk in enumerate(splits[:10], 1):
        pprint(f"\nChunk {i}:\n{chunk.page_content}")

    # OpenAI 임베딩 모델 초기화
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)
    vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
    bm25_retriever = BM25Retriever.from_documents(splits)
    faiss_retriever = vectorstore.as_retriever()

    retriever = EnsembleRetriever(
                retrievers=[bm25_retriever, faiss_retriever],
                weights=[0.5, 0.5],  # 가중치 설정 (가중치의 합은 1.0)
            )

    return retriever

In [81]:
def save_file(txt:str, file_name:str):

    with open(file_name, 'w', encoding='utf-8') as content_file:
        content_file.write(txt)

    print(f"TEXT 파일 저장 완료: {file_name}")

### FAISS, BM25 저장 후 로드 -> 성능 저하

In [82]:
# def get_retreiver(type:str, idx:int):


#     identifier = f"{type}_doc_{idx}"

#     # FAISS 로드
#     faiss_path = f"faiss_index_{identifier}"
#     embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)
#     vectorstore = FAISS.load_local(faiss_path, embeddings, allow_dangerous_deserialization=True)
#     faiss_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

#     # BM25 로드
#     bm25_path = f"bm25_documents_{identifier}.json"
#     with open(bm25_path, "r", encoding="utf-8") as f:
#         loaded_docs = [Document(**doc) for doc in json.load(f)]
#     bm25_retriever = BM25Retriever.from_documents(loaded_docs)

#     # EnsembleRetriever 재구성
#     retriever = EnsembleRetriever(
#         retrievers=[bm25_retriever, faiss_retriever],
#         weights=[0.5, 0.5],
#     )

#     return retriever


In [83]:
from langchain_core.runnables import RunnablePassthrough

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
    
# Prompt 및 Chain 구성
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = " ".join([doc.page_content for doc in inputs["context"]])
        print(f"Context output: {context_text}")
        return {"context": context_text, "quiz_list": inputs["quiz_list"]}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", """
    당신은 AI 강사입니다.
    아래의 {context} 안에서만 반드시 **한국말**로 된, 이전에 물어봤던 문제와 유사하지 않은
    단 하나의 질문을 생성해주세요.
    
    반드시 질문은 질문 내용으로 명확히 답할 수 있는 질문이어야 합니다. 
    질문을 만들 때에는 코드와 관련된 특정 동작이나 목적에 대해 물어야 하며, 질문 안에 반드시 **코드를 포함**해야 합니다.
    에시 코드는 질문에 포함하지마세요.
    
    예시:
        코드:
            dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
     
        이 코드에서 이미지 데이터를 불러올 때 한 번에 32개의 이미지를 불러오나요? (O/X)
     
    [중요]
    아래의 금지리스트들과 유사한 질문을 절대 생성하지 마시오.
    금지리스트: 이전에 만들었던 질문, "QuizList"
    
    주관적인 변수에 대한 의견을 묻는 질문은 절대 생성하지 마세요.
    예를 들어, "타이타닉 데이터셋의 'embarked' 열을 숫자로 매핑할 때, 'S'는 어떤 숫자로 매핑되나요?", "2016년 이후의 데이터를 사용했을 때 r2 score와 mse가 더 좋은 점수를 보였다 (O, X 질문)"와 같은 질문은 피해야 합니다.
     
    개발 외의 역량을 붇는 질문은 절대 생성하지 마세요.
    예를 들어, "위험중립형 투자자는 kodex골드, tiger나스닥100, kodex 삼성그룹을 각각 몇 개 매수하는 것이 추천되나요?"와 같은 질문은 피해야 합니다.
     
    질문에서 제공된 정보로 명확히 답할 수 없는 질문만 절대 생성하지 마세요.
    예를 들어, "데이터가 증가할수록 모델의 평가지표가 안 좋아지는 이유는 무엇인가요? (O, X)", "이미지 데이터셋을 불러올 때 한 번에 몇 개의 이미지를 불러오나요? (a) 16개 (b) 32개 (c) 64개 (d) 128개"와 같은 질문은 피해야 합니다.
     
    질문의 형태는 반드시 객관식 또는 OX 질문 형태여야만 합니다. (어떤, 무엇을 묻는 질문은 생성하지 마세요.)
    예를 들어, "인스턴트 초기화에 쓰이는 생성자에 __call__ 메소드를 호출하면 어떤 값이 반환되나요? (O/X)"과 같은 질문은 피해야 합니다.
    예를 들어, 주관식과 OX 질문이 결합되거나 객관식과 OX 질문이 결합된 형태는 절대 생성하지 마세요.
     
    또한, 아래의 제약 조건과 출제 방식에 맞춘 질문을 생성해주세요.
     
    제약 조건:
    1. "Context"에서 제공된 내용만 기반으로 질문을 생성하세요.
    2. AI와 관련된 질문만 생성하세요.
    3. 질문의 형태는 객관식(MCQ) 또는 O,X 형태여야 합니다.
    4. 질문은 반드시 질문 내용만 담겨야 합니다. "quiz:" 나 "질문:" 같은 불필요한 수식어구는 붙이지 마세요.
    5. 질문에서 제공된 정보로 명확히 답할 수 있는 질문만 생성하세요.

    출제 방식:
    - 질문은 반드시 객관식(MCQ) 또는 O,X 형태로 출제합니다.
    - "Context"에 명시적으로 언급된 개념, 정의, 또는 내용을 활용하세요.
    
    Context:
    {context}
    
    QuizList:
    {quiz_list}

    """)
])


In [ ]:
concept_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    당신은 AI 강사입니다.
    아래의 {context} 안에서만 반드시 **한국말**로 된 하나의 질문을 생성해주세요.
    (최대한 코드에 관한 시나리오적 질문이면 더 좋습니다.)
    {quiz_list}에 존재하는 질문들과는 최대한 덜 유사한 질문을 생성해주세요.
    아래의 제약 조건과 출제 방식에 맞춘 질문을 생성해주세요.
     
    제약 조건:
    1. "Context"에서 제공된 내용만 기반으로 질문을 생성하세요.
    2. AI 관련 내용이 아닌 질문은 생성하지 마세요
    3. "QuizList"에 이미 있는 질문과 유사하지 않은 새로운 질문을 생성하세요.

    출제 방식:
    - 질문은 반드시 보기가 있는 객관식(MCQ) 또는 O,X 형태로 출제하세요.
    - "Context"에 명시적으로 언급된 개념, 정의, 또는 내용을 활용하세요.
    - 질문은 반드시 질문 내용만 담겨야 합니다. 정답을 포함하지마세요.
    - 질문 내용에는 "quiz:" 나 "질문:" 같은 불필요한 수식어구는 담겨서는 안됩니다.
    
    Context:
    {context}
    
    QuizList:
    {quiz_list}

    """)
])

In [126]:
from langchain.schema import StrOutputParser


# RAG Chain 생성 함수
def create_rag_chain(retriever):
    return (
        {
            "context": retriever,
            "quiz_list": DebugPassThrough()
        }
        | DebugPassThrough()  # DebugPassThrough()가 실제로 어떤 역할을 하는지 확인
        | ContextToText()     # Text 변환을 위한 ContextToText
        | prompt              # prompt 사용
        | llm                 # LLM 호출
        | StrOutputParser()   # 출력 파서
    )

In [ ]:
# RAG Chain 생성 함수
def create_concept_rag_chain(retriever):
    return (
        {
            "context": retriever,
            "quiz_list": DebugPassThrough()
        }
        | DebugPassThrough()  # DebugPassThrough()가 실제로 어떤 역할을 하는지 확인
        | ContextToText()     # Text 변환을 위한 ContextToText
        | prompt             # prompt 사용
        | llm                 # LLM 호출
        | StrOutputParser()   # 출력 파서
    )

In [128]:
def choose_txt_list(type_:str):

    if type_ == "dl":
        return dl_txt_list
    if type_ == "ml":
        return ml_txt_list
    if type_ == "llm":
        return llm_txt_list
    if type_ == "python":
        return python_txt_list
    if type_ == "open_source":
        return open_source_txt_list

In [121]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def is_similar(new_quiz, quiz_list, threshold=0.8):
    vectorizer = TfidfVectorizer().fit_transform([new_quiz] + quiz_list)
    vectors = vectorizer.toarray()
    similarities = cosine_similarity(vectors[0:1], vectors[1:])
    return any(sim >= threshold for sim in similarities[0])


In [129]:

from langchain.globals import set_llm_cache, get_llm_cache
from langchain.cache import InMemoryCache

type_ = input("선택하실 AI 교재 타입 (dl, ml, llm, python, open_source) 를 입력해주세요!")
order = int(input("선택하실 AI 교쟈 챕터를 입력해주세요! 종료를 원하신다면 0을 입력해주세요!"))
# query = input("어떤 내용을 질문하고 싶으신가요?")
# set_llm_cache(InMemoryCache()) # 캐시 메모리 설정
# llm_cache = get_llm_cache()
j = 0
valid_type = ["dl", "ml", "llm", "python", "open_source"]
quiz_list = []
txt_list = choose_txt_list(type_)
current_index = 0

while True: 

    if (order == 0):
        break

    elif (type_ not in valid_type):
        type_ = input("교재 타입을 반드시!(dl, ml, llm, python, open_source) 안에서 입력해주세요!")
        order = int(input("선택하실 딥러닝 챕터의 교재 번호를 입력해주세요! 종료를 원하신다면 0을 입력해주세요!"))
        continue

    elif (type_ in valid_type):
        break

while True: 
        
    quiz_list = quiz_list[-5:]  # 최신 5개 퀴즈만 보관
    quiz_str = " ".join(quiz_list)
    # retriever = get_retriever(txt_list[order-1])
    retriever = get_retriever(txt_list[order-1], current_index)
    rag_chain = create_concept_rag_chain(retriever)


    query = concept_prompt.format(
        context=retriever,
        quiz_list=quiz_list,       
    )


    try:
        response = rag_chain.invoke("퀴즈 하나를 생성해줘")
    except Exception as e:
        print(f"Quiz 생성 중 오류 발생: {e}")
        continue
    
    if len(quiz_list) > 0 and is_similar(response, quiz_list, 0.7):
        continue

    quiz = response
    quiz_list.append(quiz)
    print("quiz_list의 크기: ", len(quiz_list))

    print(f"Quiz : {quiz}")
    j+=1
    save_file(''.join(quiz), f"quiz_list_{j}.txt")
    
    
    # 2. 사용자 답변 수집
    user_answer = input("답변을 입력하세요 힌트를 원한다면 help를, 종료를 원하시면 exit을 입력해주세요.: ").strip()


    if user_answer.strip().lower() == "exit":
        print("대화를 종료합니다.")
        break

    elif user_answer.strip().lower() == "help":
        
    
    if not user_answer:
        print("답변이 비어 있습니다. 다시 입력해주세요.")
        continue

    # 3. 사용자 답변에 대한 피드백 생성
    # 3. 사용자 답변에 대한 피드백 생성
    feedback_prompt = ChatPromptTemplate.from_messages([
        ("system", f"""
        AI 강사로서 다음 퀴즈의 정답 여부를 확인하고 피드백을 제공하세요.
        피드백은 아래와 같은 형식이어야 합니다:
        
        - 정답 여부: "N번" 또는 "예/아니오"
        - 추가 설명: (정답과 관련된 추가 정보를 제공하세요)
        
        퀴즈 : {{quiz}}
        답변 : {{user_answer}}
        
        """)
    ])

    feedback_chain = feedback_prompt | llm
    feedback = feedback_chain.invoke({"quiz": quiz, "user_answer": user_answer})
    print("Feedback:")
    print(feedback)
    current_index += 5

Top 10 chunks:
('\n'
 'Chunk 1:\n'
 '[스파르타코딩클럽] 5. 합성곱 신경망(CNN)📘[SCC] 기초가 탄탄한 딥러닝/📚[스파르타코딩클럽] 기초가 탄탄한 딥러닝 - '
 '2주차/📕[스파르타코딩클럽] 5. 합성곱 신경망(CNN)Made with📕[스파르타코딩클럽] 5. 합성곱 신경망(CNN)[수업 '
 '목표]합성곱 신경망의 개념에 대해서 배워보고 어떤 원리로 동작하는지 알아봅시다Pytorch로 간단한')
('\n'
 'Chunk 2:\n'
 '알아봅시다Pytorch로 간단한 CNN 모델 구현 실습을 진행해 봅시다[목차]01. CNN의 기본 구조와 동작 원리02. 실습: CNN을 '
 '이용한 이미지 분류 (PyTorch) - 입력 이미지에 필터(커널)를 적용하여 특징 맵(feature map)을 생성합니다. - 입력 '
 '이미지에 필터(커널)를 적용하여 특징 맵(feature map)을 생성합니다.\ufeff')
('\n'
 'Chunk 3:\n'
 '- 필터는 이미지의 국소적인 패턴을 학습합니다. - 필터는 이미지의 국소적인 패턴을 학습합니다.\ufeff\n'
 '풀링 층 (Pooling Layer)\n'
 ' - 특징 맵의 크기를 줄이고, 중요한 특징을 추출합니다. - 특징 맵의 크기를 줄이고, 중요한 특징을 추출합니다.\ufeff')
('\n'
 'Chunk 4:\n'
 '- 주로 Max Pooling과 Average Pooling이 사용됩니다. - 주로 Max Pooling과 Average Pooling이 '
 '사용됩니다.\ufeff\n'
 '완전 연결 층 (Fully Connected Layer)\n'
 ' - 추출된 특징을 바탕으로 최종 예측을 수행합니다. - 추출된 특징을 바탕으로 최종 예측을 수행합니다.\ufeff')
('\n'
 'Chunk 5:\n'
 '- CNN이라는 분석레이어를 통해 추출한 특성을 바탕으로 결론을 내리는 부분 - CNN이라는 분석레이어를 통해 추출한 특성을 바탕으로 '
 '결론을 내리는 부

In [ ]:
from langchain.globals import set_llm_cache, get_llm_cache
from langchain.cache import InMemoryCache

type_ = input("선택하실 딥러닝 챕터의 교재 타입 (dl, ml, llm, python, open_source) 를 입력해주세요!")
order = int(input("선택하실 딥러닝 챕터의 교재 번호를 입력해주세요! 종료를 원하신다면 0을 입력해주세요!"))
# query = input("어떤 내용을 질문하고 싶으신가요?")
# set_llm_cache(InMemoryCache()) # 캐시 메모리 설정
# llm_cache = get_llm_cache()
j = 0
valid_type = ["dl", "ml", "llm", "python", "open_source"]
quiz_list = []
txt_list = choose_txt_list(type_)
current_index = 0

while True: 

    if (order == 0):
        break

    elif (type_ not in valid_type):
        type_ = input("교재 타입을 반드시!(dl, ml, llm, python, open_source) 안에서 입력해주세요!")
        order = int(input("선택하실 딥러닝 챕터의 교재 번호를 입력해주세요! 종료를 원하신다면 0을 입력해주세요!"))
        continue

    elif (type_ in valid_type):
        break

while True: 
        
    quiz_list = quiz_list[-5:]  # 최신 5개 퀴즈만 보관
    quiz_str = " ".join(quiz_list)
    retriever = get_retriever(txt_list[order-1])
    # retriever = get_retriever(txt_list[order-1], current_index)
    rag_chain = create_concept_rag_chain(retriever)

    try:
        response = rag_chain.invoke("질문 하나를 생성해주세요.")
    except Exception as e:
        print(f"Quiz 생성 중 오류 발생: {e}")
        continue
    
    if len(quiz_list) > 0 and is_similar(response, quiz_list, 0.5):
        continue

    quiz = response
    quiz_list.append(quiz)
    print("quiz_list의 크기: ", len(quiz_list))

    # discription_prompt = ChatPromptTemplate.from_messages([
    #     ("system", f"""
    #     quiz에 대해 정답을 찾을 수 있는 부분을 context에서만 찾아서 한국말로 보여주세요.
    #     되도록이면 context 중에서도 코드 부분을 보여주세요.
    #     단, quiz와는 내용이 정확하게 일치하는 부분은 제외해주세요.
         
    #     [주의]
    #     코드외의 정답에 대한 직접적인 설명적인 힌트는 포함시키지 마세요.
        
    #     quiz : {{quiz}}
    #     context : {{context}}
        
    #     """)
    # ])

    # discription_chain = discription_prompt | llm
    # discription = discription_chain.invoke({"quiz": quiz, "context": retriever})

    print(f"Quiz : {quiz}")
    j+=1
    save_file(''.join(quiz), f"quiz_list_{j}.txt")
    # save_file(''.join([discription.content, str(quiz)]), f"quiz_list_{j}.txt")
    
    
    # 2. 사용자 답변 수집
    user_answer = input("답변을 입력하세요 종료를 원하시면 exit을 입력해주세요.: ").strip()

    if user_answer.strip().lower() == "exit":
        print("대화를 종료합니다.")
        break
    
    if not user_answer:
        print("답변이 비어 있습니다. 다시 입력해주세요.")
        continue

    # 3. 사용자 답변에 대한 피드백 생성
    # 3. 사용자 답변에 대한 피드백 생성
    feedback_prompt = ChatPromptTemplate.from_messages([
        ("system", f"""
        AI 강사로서 다음 퀴즈의 정답 여부를 확인하고 피드백을 제공하세요.
        피드백은 아래와 같은 형식이어야 합니다:
        
        - 정답 여부: "N번" 또는 "예/아니오"
        - 추가 설명: (정답과 관련된 추가 정보를 제공하세요)
        
        퀴즈 : {{quiz}}
        답변 : {{user_answer}}
        
        """)
    ])

    feedback_chain = feedback_prompt | llm
    feedback = feedback_chain.invoke({"quiz": quiz, "user_answer": user_answer})
    print("Feedback:")
    print(feedback)
    current_index += 5